<a href="https://colab.research.google.com/github/vasudev-sharma/Flixstock-assignment/blob/master/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the data
!FILEID='1p7-dU6rDuqZ2mxv5ac5AWndt4z19aS6j' && \
FILENAME='data.zip' && \
FILEDEST="https://docs.google.com/uc?export=download&id=${FILEID}" && \
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate ${FILEDEST} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=${FILEID}" -O $FILENAME && rm -rf /tmp/cookies.txt

--2021-06-14 09:40:39--  https://docs.google.com/uc?export=download&id=1p7-dU6rDuqZ2mxv5ac5AWndt4z19aS6j
Resolving docs.google.com (docs.google.com)... 74.125.23.101, 74.125.23.113, 74.125.23.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.23.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

-                       [ <=>                ]   3.22K  --.-KB/s    in 0s      

2021-06-14 09:40:39 (17.5 MB/s) - written to stdout [3300]

--2021-06-14 09:40:39--  https://docs.google.com/uc?export=download&confirm=onsd&id=1p7-dU6rDuqZ2mxv5ac5AWndt4z19aS6j
Resolving docs.google.com (docs.google.com)... 108.177.97.138, 108.177.97.100, 108.177.97.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.97.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-98-docs.googleusercontent.com/docs/securesc/9picvh6kpg53mnro9gstugs352b2ijck/du58s60ued95hh

In [2]:
# unzip the data
%%bash
unzip data.zip -d data


Archive:  data.zip
   creating: data/classification-assignment/
  inflating: data/__MACOSX/._classification-assignment  
   creating: data/classification-assignment/images/
  inflating: data/__MACOSX/classification-assignment/._images  
  inflating: data/classification-assignment/attributes.csv  
  inflating: data/__MACOSX/classification-assignment/._attributes.csv  
  inflating: data/classification-assignment/images/11523001185834-Masch-Sports-Mens-Polyester-Printed--Colourblocked-T-Shirts---Pack-of-2-8971523001185724-4.jpg  
  inflating: data/__MACOSX/classification-assignment/images/._11523001185834-Masch-Sports-Mens-Polyester-Printed--Colourblocked-T-Shirts---Pack-of-2-8971523001185724-4.jpg  
  inflating: data/classification-assignment/images/11491549671402-PUMA-White-FIGC-Italia-Away-Polyester-T-shirt-7541491549671229-3.jpg  
  inflating: data/__MACOSX/classification-assignment/images/._11491549671402-PUMA-White-FIGC-Italia-Away-Polyester-T-shirt-7541491549671229-3.jpg  
  inflat

In [103]:

# PyTorch imports
import torch
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import os
from pathlib import Path
import pandas as pd
from PIL import Image
import numpy as np
import math
import random

In [104]:
!pip install wandb
import wandb


In [105]:
data_path = Path('/content/data/classification-assignment')

In [106]:
# transforms 
train_transforms =  transforms.Compose(
    [
     transforms.Resize((256, 256)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
      )

valid_transforms = transforms.Compose(
          [transforms.Resize((256, 256)),
           transforms.ToTensor(),
           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           ]
                                      )


In [107]:
class FashionDataset(Dataset):
  def __init__(self, root, transform=None, target_transform=None):
    self.root = root
    self.transform = transform
    self.target_transform = target_transform
    self.samples = os.listdir(str(self.root / 'images'))

    # Attributes dataframe
    self.df_attributes = pd.read_csv(str(self.root / 'attributes.csv'))

    # one hot encode the target attributes
    self.preprocess_targets() 


  def __len__(self):
    return len(self.samples)
  
  def __getitem__(self, index):

    # shuffle the images list
    np.random.shuffle(self.samples)

    # name of the image
    filename_image = self.samples[index]

    
    try:
      image = Image.open(str(self.root / 'images' / filename_image)).convert('RGB')
    except Exception as e:
      print('Path of the image is', str(self.root / 'images' / filename_image))
      print('Unable to read the image')

    # retreive the specific row of given index
    df_row = self.df_attributes.loc[self.df_attributes['filename'] == filename_image]

    if self.transform is not None:
      image = self.transform(image)

    '''if self.target_transform is not None:
      target = self.target_transform(target)
    '''
    # Target
    target_start_idx = self.df_attributes.columns.get_loc('neck_0.0')
    target = torch.tensor(self.df_attributes.iloc[0].tolist()[target_start_idx:], dtype=torch.float32)

    # return target and labels
    return image, target
  
  def preprocess_targets(self):
    # Drop rows which have na
    self.df_attributes.dropna()

    # one hot encode the Neck attribute
    one_hot_neck = pd.get_dummies(self.df_attributes.neck, prefix='neck')

    # one hot encode the sleeve_length attribute
    one_hot_sleeve_length = pd.get_dummies(self.df_attributes.sleeve_length, prefix='sleeve_length')

    # one hot encode the patter attribute
    one_hot_pattern = pd.get_dummies(self.df_attributes.pattern, prefix='pattern')

    # concatenate the one hot encoded attributes to dataframe
    self.df_attributes = pd.concat([self.df_attributes, one_hot_neck, one_hot_sleeve_length, one_hot_pattern], axis=1)


  


In [108]:
dataset = FashionDataset(data_path, transform=train_transforms)

In [109]:
# split of dataset into train, valid, test datasets

train_split = 0.75
val_split = 0.15
test_split = 1 - (train_split + val_split)

len_train_dataset = math.ceil(train_split * len(dataset))
len_val_dataset = math.ceil(val_split * len(dataset))
len_test_dataset = len(dataset) - (len_train_dataset + len_val_dataset)




In [110]:
ds_train, ds_valid, ds_test = random_split(dataset, [len_train_dataset, len_val_dataset, len_test_dataset])

In [111]:
# sanity check 
assert len(ds_train) == len_train_dataset
assert len(ds_valid) == len_val_dataset
assert len(ds_test) == len_test_dataset

In [112]:
# hyperparameters
batch_size = 16
lr = 0.001
epochs=30
log_freq = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

n_labels=21
model_name = 'resnet50'


In [113]:
device

device(type='cuda', index=0)

In [114]:
# Dataloaders
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dl_valid = DataLoader(ds_valid, batch_size=batch_size, shuffle=True, pin_memory=True)
dl_test = DataLoader(ds_valid, batch_size=len(ds_test), shuffle=False, pin_memory=True)

In [115]:
# sanity check the shape 
for batch in dl_train:
  img_batch, target_batch = batch
  print(img_batch.shape)
  print(target_batch.shape)
  break

torch.Size([16, 3, 256, 256])
torch.Size([16, 21])


In [116]:
#@title
!wandb login 202040aaac395bbf5a4a47d433a5335b74b7fb0e

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Model Training

In [136]:
# Pretrained ResNet model
model = torch.hub.load('pytorch/vision:v0.9.0', model_name, pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [137]:
model.fc = nn.Linear(2048, n_labels)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [138]:
model.loss_func = nn.BCELoss()
model.optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [139]:
from sklearn.metrics import *

In [140]:
def calculate_metrics(pred, target, threshold=0.5):
  pred = np.array(pred > threshold, dtype = float)
  return { 
      # Micro scores
      'micro/precision':precision_score(y_true=target, y_pred=pred, average='micro'),
      'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
      'micro/f1_score': f1_score(y_true=target, y_pred=pred, average='micro'),

      # Macro scores
      'macro/precision':precision_score(y_true=target, y_pred=pred, average='macro'),
      'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
      'macro/f1_score': f1_score(y_true=target, y_pred=pred, average='macro'),

  }

In [141]:
'''
def train_step(features, labels):

  # For training
  model.train()

  model.optimizer.zero_grad()

  # Forward pass
  logits = model(features)
  preds = torch.sigmoid(logits)

  # loss
  loss = model.loss_func(preds.type(torch.float), labels)
  # metrics
  metrics_dict = calculate_metrics(preds, targets)

  # backward pass
  model.loss.backward()

  # update weights
  model.optimizer()

  return loss.item(), metrics_dict

'''

'\ndef train_step(features, labels):\n\n  # For training\n  model.train()\n\n  model.optimizer.zero_grad()\n\n  # Forward pass\n  logits = model(features)\n  preds = torch.sigmoid(logits)\n\n  # loss\n  loss = model.loss_func(preds.type(torch.float), labels)\n  # metrics\n  metrics_dict = calculate_metrics(preds, targets)\n\n  # backward pass\n  model.loss.backward()\n\n  # update weights\n  model.optimizer()\n\n  return loss.item(), metrics_dict\n\n'

In [142]:
'''
def valid_step(features, labels):
  
  # validation
  model.eval()

  with torch.no_grad():
    # Forward pass
    logits = model(features)
    preds = torch.sigmoid(logits)

    # Loss
    loss = model.loss_func(preds.type(torch.float), labels)
    # metrics
    metrics_dict = calculate_metrics(preds)
'''

'\ndef valid_step(features, labels):\n  \n  # validation\n  model.eval()\n\n  with torch.no_grad():\n    # Forward pass\n    logits = model(features)\n    preds = torch.sigmoid(logits)\n\n    # Loss\n    loss = model.loss_func(preds.type(torch.float), labels)\n    # metrics\n    metrics_dict = calculate_metrics(preds)\n'

In [143]:
def train_model(model, dl_train, dl_valid, epochs, log_freq):
  print('***************Training of the model has started**************')
  wandb.watch(model, log='all')

  iteration = 0
  for epoch in range(epochs+1):

    batch_losses = []

    for batch_idx, (features, targets) in enumerate(dl_train, 1):
      model.train()
      
      model.optimizer.zero_grad()

      # Migrate the features and targets to device
      features, targets = features.to(device), targets.to(device)

      # Forward pass
      logits = model(features)
      preds = torch.sigmoid(logits)

      # loss
      loss = model.loss_func(preds.type(torch.float), targets)

      # backward pass
      loss.backward()

      # update weights
      model.optimizer.step()

      batch_losses.append(loss)

      if iteration % log_freq ==0:
        # Validation 
        model.eval()
        with torch.no_grad():

          preds_list = []
          targets_list = []
          for val_batch_idx, (val_features, val_targets) in enumerate(dl_valid, 1):
              
              val_features = val_features.to(device)
              
              # Forward pass
              val_logits = model(val_features)
              val_preds = torch.sigmoid(val_logits)

              preds_list.extend(val_preds.cpu().numpy())
              targets_list.extend(val_targets.cpu().numpy())

        
        metrics_dict = calculate_metrics(np.array(preds_list), np.array(targets_list))
        print('[Epoch = % d]:  micro/precision = %.3f, micro/recall = %.3f, micro/f1_score = %.3f, macro/precision = %.3f, macro/recall = %.3f, macro/f1_score = %.3f' \
              % (epoch, metrics_dict['micro/precision'], metrics_dict['micro/recall'], metrics_dict['micro/f1_score'], metrics_dict['macro/precision'], metrics_dict['macro/recall'], metrics_dict['macro/f1_score']))

        print('====='*8)


      iteration+=1

  loss_mean = np.mean(batch_losses)
  print('\n')
  print("epoch:{:2d} iter:{:3d} train: loss:{:.3f}".format(epoch, iteration, loss_mean))

      





In [144]:
def main():
  # wandb initialize a new run
  wandb.init(project='Flixstock-assignment')
  wandb.watch_called = False

  config = wandb.config
  config.batch_size = batch_size
  config.epochs = epochs
  config.lr = lr
  config.seed = 42
  config.labels = n_labels
  config.device = device
  config.model_name = model_name
  config.log_freq = log_freq

  # set seed and set cuddn to deterministic for reproducible results
  torch.manual_seed(config.seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  np.random.seed(config.seed)
  random.seed(config.seed)

  train_model(model, dl_train, dl_valid, epochs, log_freq)

 
if __name__=='__main__':
  main()
  wandb.finish()

***************Training of the model has started**************


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


[Epoch =  0]:  micro/precision = 0.643, micro/recall = 0.989, micro/f1_score = 0.779, macro/precision = 0.095, macro/recall = 0.094, macro/f1_score = 0.095
Path of the image is /content/data/classification-assignment/images/Thumbs.db
Unable to read the image


UnboundLocalError: ignored